In [20]:
import numpy as np

total_warehouses = 14
total_weeks = 4

storage_capacity = np.array([28,23,7,15,31,38,56,7,27,3,35,8,23,7])
terminal_capacity = np.array([2,2,1,2,2,2,2,1,1,1,2,2,2,1])           
max_allottment = np.array([8,8,8,8,8,8,8,8,8,8,8,8,8,8])

rake_penalty_h =20;
rake_penalty_f = 50;

max_iter = 100
k_shift = 135
k_reset = 575
k_terminate = 7450
comb_matrix =    np.array([0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0,1,0,1,0,0,0,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,1,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,0,1,1,1,1,1,0,0,1,0,1,1,1,1,1,0,1,1,1,1,0,0,0,0,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,1,0,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,1,0,0,0,0,0,0,0,0,1,1,1,1,1,0])  
comb_matrix = comb_matrix.reshape(14,14)

In [21]:
def capacity_utilization_pf(a,storage_capacity):
    n = np.ma.size(a)
    b = np.zeros((n))
    for i in range(0,n):
        b[i] = storage_capacity[i]/(a[i])
        b[i] = 10*b[i]
    return b

In [22]:
def surplus_demand(surplus,initial_stock_level,demand,max_allottment,available_space,total_warehouses,total_weeks,storage_capacity,random_array,random_index):
    dv1 = random_array[random_index]
    random_index += 1
    while (surplus>0 and dv1<(total_warehouses*total_weeks/2)):
        max_cupf = capacity_utilization_pf(initial_stock_level+demand,storage_capacity)
        available = np.minimum(max_allottment,available_space) - demand
        max_cupf_ind= max_cupf.argmax()
        max_possible_addition = min(available[max_cupf_ind],surplus)
        addition = (random_array[random_index]%max_possible_addition) + 1
        random_index += 1
        demand[max_cupf_ind] += addition
        surplus -= addition
    while (surplus>0 and dv1>=(total_warehouses*total_weeks/2)):
        available = np.minimum(max_allottment,available_space) - demand
        rand = 0
        selected = 0
        while (selected == 0):
            rand = random_array[random_index]%total_warehouses
            random_index += 1
            selected = available[rand]
        max_possible_addition = min(available[rand],surplus)
        addition = (random_array[random_index]%max_possible_addition) + 1
        random_index += 1
        demand[rand] += addition
        surplus -= addition
    return demand,random_index

In [23]:
def surplus_allocation(rake_allocated,demand,total_weeks,random_array,random_index):
    allocated1 = np.sum(rake_allocated)
    allocated = np.sum(allocated1)
    excess = np.array(np.nonzero(demand>4))
    #num = np.size(excess)
    #excess = excess.reshape(num,)
    for i in excess[0]:
        dem = demand[i]
        while(dem > 8-demand[i]):
            w = random_array[random_index]%4
            random_index += 1
            if(rake_allocated[i][w]==0):
                rake_allocated[i][w] += 2
                allocated += 2
                dem -= 2
    return allocated,rake_allocated,random_index

In [24]:
def rake_allocation(allocated,total_rakes,rake_allocated,monthly_allotment,comb_matrix,comb,k_shift,k_reset,k_terminate,terminal_capacity,total_warehouses,total_weeks,random_array,random_index):
    k = 1
    while allocated<total_rakes:
        j = random_array[random_index]%total_warehouses
        random_index += 1
        w = random_array[random_index]%total_weeks
        random_index += 1
        #selected_row = rake_allocated[j,:]
        sum4 = sum(rake_allocated[j])
        dv2 = random_array[random_index]
        random_index += 1
        partition = 500
        iteration = 1;
        while (sum4<monthly_allotment[j]) and (iteration<2):
            if(rake_allocated[j][w]==0):
                #comb_row = comb_matrix[j,:]
                #comb_row_sum = sum(comb_row)
                combwh = np.array(np.nonzero(comb_matrix[j]==1))
                sum_ra = np.sum(rake_allocated,axis=1)
                remaining_aloc=monthly_allotment-sum_ra
                c=np.array(np.nonzero(remaining_aloc[combwh[0]]>0))
                possible_wh=combwh[0,c]
                num = np.size(possible_wh)
                if(num>0):
                    #combwh = np.array(np.nonzero(comb_row==1))
                    #num = np.size(combwh)
                    #combwh = combwh.reshape(num,)
                    rv = random_array[random_index]%num
                    random_index += 1
                    cw = possible_wh[0,rv]
                    #selected_row2 = rake_allocated[cw,:]
                    #sum6 = sum(selected_row2)
                    sum6 = sum_ra[cw]
                    while(rake_allocated[cw][w]==0 and sum6<monthly_allotment[cw] and dv2<partition):
                        rake_allocated[j][w] += 1
                        rake_allocated[cw][w] += 1
                        comb[j][cw][w] += 1
                        comb[cw][j][w] += 1
                        allocated += 2
                        sum4 += 1
                        sum6 += 1
                    count = 1
                    while(dv2>=partition and k>k_shift and count<2 and (sum4+1)<monthly_allotment[j]):
                        if(terminal_capacity[j]==2 and rake_allocated[j][w]==0):
                            if(sum4<(monthly_allotment[j]-1)):
                                rake_allocated[j][w] += 2
                                allocated += 2
                                sum4 += 2
                        count+=1
                elif(num == 0 and (sum4+1)<monthly_allotment[j]):
                    rake_allocated[j][w] += 2
                    allocated += 2
                    sum4 += 2
            iteration+=1
        if(k%k_reset == 0):
            rake_allocated = np.zeros((total_warehouses,total_weeks))
            comb = np.zeros((total_warehouses,total_warehouses,4))
            allocated,rake_allocated,random_index = surplus_allocation(rake_allocated,monthly_allotment,total_warehouses,random_array,random_index)
        if(k>k_terminate):
            break
        k += 1
    return rake_allocated,comb,allocated,random_index

In [25]:
def penalty(rake_allocated,rake_penalty_h,rake_penalty_f,initial_stock_level,demand,weekly_penalty,storage_capacity):
    half_rakes = np.array(np.nonzero(rake_allocated==1))
    full_rakes = np.array(np.nonzero(rake_allocated==2))
    size1 = np.size(half_rakes)
    size2 = np.size(full_rakes)
    cost1 = size1*rake_penalty_h/2
    cost2 = size2*rake_penalty_f/2
    rake_penalty = cost1+cost2
    alloted_weeks = np.array(np.nonzero(rake_allocated>0))
    week_penalty = 0
    weeknum = np.size(alloted_weeks)/2
    m = 0
    while m < weeknum:
        swh = alloted_weeks[0,m]
        sw = alloted_weeks[1,m]
        week_penalty += weekly_penalty[swh][sw]
        m += 1

    capacity_utilization_penalty = np.sum(capacity_utilization_pf(initial_stock_level+demand,storage_capacity))
    total_penalty = rake_penalty + week_penalty + capacity_utilization_penalty
    return total_penalty

In [31]:
import time

def simulate(iteration):
    start = time.time()
    first = 1
    total_rakes = 30
    demand = np.array([3,0,1,5,5,2,1,2,3,2,0,1,1,0])
    original_demand = demand
    initial_stock_level = np.array([22,18,6,10,23,26,39,5,10,2,17,7,14,2])

    weekly_penalty =    np.array([10,1,1,1,10,1,1,1,10,1,1,2,10,1,1,1,2,2,1,1,10,1,1,1,3,1,1,1,5,2,1,4,5,2,1,1,2,2,1,1,10,1,1,4,4,2,1,2,4,1,1,2,3,2,1,1])
    weekly_penalty = weekly_penalty.reshape(total_warehouses,total_weeks)

    random_array = np.random.randint(0,1000,10000*iteration)
    random_index = 0
    for i in range(0,iteration):
        demand = original_demand
        rake_allocated = np.zeros((total_warehouses,total_weeks))
        total_demand = np.sum(demand)
        surplus = total_rakes - total_demand
        available_space = storage_capacity - initial_stock_level
        demand,random_index = surplus_demand(surplus,initial_stock_level,demand,max_allottment,available_space,total_warehouses,total_weeks,storage_capacity,random_array,random_index)
        comb = np.zeros((total_warehouses,total_warehouses,total_weeks))
        monthly_allotment = demand
        allocated,rake_allocated,random_index = surplus_allocation(rake_allocated,demand,total_weeks,random_array,random_index)
        rake_allocated,comb,allocated,random_index = rake_allocation(allocated,total_rakes,rake_allocated,monthly_allotment,comb_matrix,comb,k_shift,k_reset,k_terminate,terminal_capacity,total_warehouses,total_weeks,random_array,random_index)
        total_penalty = penalty(rake_allocated,rake_penalty_h,rake_penalty_f,initial_stock_level,demand,weekly_penalty,storage_capacity)
        if(first ==1 and allocated == total_rakes):
            final_weekly_distribution = rake_allocated
            final_total_penalty = total_penalty
            final_comb = comb
            first += 1
        if(allocated == total_rakes and first != 1):
            if(total_penalty<final_total_penalty):
                final_weekly_distribution = rake_allocated
                final_total_penalty = total_penalty
                final_comb = comb
    print(final_weekly_distribution)
    print(final_total_penalty)

    end = time.time()
    print(end-start)

In [27]:
import cProfile

In [32]:
cProfile.run('simulate(100)')

[[ 0.  2.  1.  2.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 2.  1.  1.  1.]
 [ 1.  1.  2.  1.]
 [ 0.  1.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  1.]
 [ 1.  2.  1.  1.]
 [ 0.  1.  1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  0.]]
895.212573918
1.869023323059082
         386366 function calls (386352 primitive calls) in 1.870 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       57    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:416(parent)
       57    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
      100    0.003    0.000    0.004    0.000 <ipython-input-21-e8b393e239de>:1(capacity_utilization_pf)
      100    0.038    0.000    0.038    0.000 <ipython-input-22-5491752f4d67>:1(surplus_demand)
      340    0.013    0.000    0.023    0.000 <ipython-input-23-2d028b9e7752>:1(surplus_allocation)
      100    0.982    0.010    1.791  